In [3]:
%pip install praw

     -------------------------------------- 189.4/189.4 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
import praw
import pandas as pd

def reddit_pull(subreddit, tag):
    # Create a Reddit instance
    reddit = praw.Reddit(
        client_id='7RXgU0jwE9m8yHjy4rPpng',
        client_secret='1eiT64G_imzQdjaS357idybjMedozw',
        user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    )

    # Choose the subreddit you want to pull posts from
    subreddit = reddit.subreddit(subreddit)

    # Get the top 1000 posts from the subreddit
    new_posts = subreddit.new(limit=1000)
    hot_posts = subreddit.hot(limit=1000)

    # Create lists to store post information
    titles = []
    upvotes = []
    num_comments = []

    # Iterate over the posts and retrieve desired information
    for post in new_posts:
        titles.append(post.title)
        upvotes.append(post.score)
        num_comments.append(post.num_comments)
    
    for post in hot_posts:
        titles.append(post.title)
        upvotes.append(post.score)
        num_comments.append(post.num_comments)
        
    # Create a dataframe from the lists
    data = {
        'Title': titles,
        'Upvotes': upvotes,
        'Number of Comments': num_comments
    }
    df = pd.DataFrame(data)
    df['Political Lean'] = tag
    return df

In [13]:
subreddit_list = ['Conservative', 'Libertarian', 'Socialism', 'Progressive', 'Democrats', 'Liberal', 'Republican', 'Democratic Socialism', 'Anarcho Capitalism']
tag_list = ['Conservative', 'Conservative', 'Liberal', 'Liberal', 'Liberal', 'Liberal', 'Conservative', 'Liberal', 'Conservative']

def new_data(subreddits, tags):
    for i in range(len(subreddit_list)):
        if i == 0:
            try: 
                df = reddit_pull(subreddit_list[i], tag_list[i])
            except:
                continue
        else: 
            try:
                new_df = reddit_pull(subreddit_list[i], tag_list[i])
                df = pd.concat([df, new_df],axis=0)
            except:
                continue
    return df


In [15]:
df = new_data(subreddit_list, tag_list)

In [20]:
df = df.rename({'Upvotes':'Score','Number of Comments':'Num of Comments'},axis=1)

In [25]:
df.drop_duplicates(inplace=True)

In [27]:
df.to_csv('../data/6_14_pull.csv', index=False)